# A Test for the Tab App

This uses [importnb](https://pypi.org/project/importnb) to load the
[Tab App Example](./Tab%20App%20Example.ipynb) for interactive and automated testing.

In [ ]:
import json
import os
import time
from pathlib import Path

import importnb
from rdflib import Graph
from requests_cache import CachedSession

with importnb.Notebook():
    try:
        from Tab_App_Example import W, tabs
    except ImportError:
        from .Tab_App_Example import W, tabs

In [ ]:
pl = tabs.file_manager.loader
q = tabs.query

This notebook is used as part of the `ipyradiant` test suite, where `IPYRADIANT_TESTING`
will be set, which will trigger the automted section below.

In [ ]:
IS_TESTING = json.loads(os.environ.get("IPYRADIANT_TESTING", "false"))
IS_TESTING

Tests are implemented as buttons you can click.

In [ ]:
tests = [
    W.Button(description=f"{k}")
    for k, v in pl.file_picker.options.items()
    if v and "tree" not in k
]

In [ ]:
timings = {}


def timestamp(key, msg):
    if key not in timings:
        timings[key] = []
    timings[key] += [time.time()]
    delta = 0 if len(timings[key]) == 1 else timings[key][-1] - timings[key][-2]
    print(f"[{key}]", f"+{int(delta)}", msg)

In [ ]:
TEST_QUERY = """\
SELECT DISTINCT ?s ?p ?o
WHERE {
    ?s ?p ?o .
}
LIMIT 5
"""

In [ ]:
def _run_test(btn):
    p = btn.description

    try:
        timestamp(p, "starting...")
        tabs.graph = Graph()
        timestamp(p, "cleaned...")
        tabs.selected_index = 0
        timestamp(p, f"loading...")
        pl.file_picker.value = pl.file_picker.options[p]
        timestamp(p, f"... {len(pl.graph)} triples loaded")
        assert len(pl.graph)
        tabs.selected_index = 1
        timestamp(p, "querying...")
        q.query = TEST_QUERY
        q.run_button.click()
        assert len(q.query_result) > 0, "Failed to execute query."
        timestamp(p, "OK!")
    except Exception as err:
        timestamp(p, "ERROR")
        timestamp(p, err)
        raise Exception(f"{p} failed") from err


[d.on_click(_run_test) for d in tests]

## Show the Test Application

In [ ]:
W.VBox([tabs, W.HBox([W.Label("Start Test"), *tests])])

## (Maybe) Run the test automatically

In [ ]:
if IS_TESTING:
    for test in tests:
        test.click()